# DBS-Pro Analysis Report

### Imports

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from dbspro.cli.config import load_yaml, print_construct

## Run info
### Sample info

In [ ]:
df = pd.read_csv("samples.tsv", sep="\t").set_index("Sample")
df.style.bar(subset=["Reads"], vmin=0)

### Configs

In [ ]:
data, _ = load_yaml("dbspro.yaml")
df = pd.DataFrame.from_dict(data, orient="index", columns=["Value"])
df.index.name = "Parameter"
display(df)

### Construct

In [ ]:
print_construct("dbspro.yaml")


## Dataprocessing

### Data loading

In [ ]:
dtypes = {
    "Barcode": "object",
    "Target": "object",
    "UMI": "object",
    "ReadCount": int,
    "Sample": "category"
}
data_raw = pd.read_csv("data.tsv.gz", sep="\t", dtype=dtypes)
data_raw.head()

In [ ]:
labels = list(set(data_raw["Sample"]))
labels.sort(reverse=True)
nr_cols = 4 if len(labels) > 4 else len(labels)

### Overall QC

In [ ]:
d = data_raw.groupby("Sample", as_index=False)["ReadCount"].sum()
d["ReadCount"] /= 1_000_000
ax = sns.barplot(data=d, y="Sample", x="ReadCount", order=labels)
_ = ax.set_xlabel("Reads (M)")
_ = ax.set_title("Nr reads per Sample")

In [ ]:
d = data_raw.groupby("Sample", as_index=False)["UMI"].count()
d["UMI"] /= 1_000
ax = sns.barplot(data=d, y="Sample", x="UMI", order=labels)
_ = ax.set_xlabel("UMIs (k)")
_ = ax.set_title("Nr UMIs per Sample")

In [ ]:
d = data_raw.groupby("Sample").agg({"Barcode":"nunique"})
d["Barcode"] /= 1_000
ax = sns.barplot(data=d, y=d.index, x="Barcode", order=labels)
_ = ax.set_xlabel("Barcodes (k)")
_ = ax.set_title("Nr Barcodes per Sample")

In [ ]:
d = data_raw.groupby("Sample").agg({"Target":"nunique"})
ax = sns.barplot(data=d, y=d.index, x="Target", order=labels)
_ = ax.set_xlabel("Targets")
_ = ax.set_title("Nr Targets per Sample")

In [ ]:
d = data_raw.groupby(["Sample", "Target"], as_index=False, observed=True).agg({"UMI":"count"})
d["UMI"] /= 1000
g = sns.catplot(data=d, y="Sample", x="UMI",  col="Target", col_wrap=4, kind="bar")
g.fig.subplots_adjust(top=0.85)
_ = g.fig.suptitle("Nr UMIs per Targets")
_ = g.set_axis_labels("UMIs (k)", "")

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.set_title("UMI count distribution")
for label in labels:
    temp = data_raw[data_raw["Sample"] == label].groupby("Barcode", as_index=False)["UMI"].count().sort_values(by="UMI",ascending=False).reset_index(drop=True)
    try:
        temp.plot(ax=ax, y="UMI", logx=True, logy=True, label=label)
    except TypeError:
        pass
    
ax.set_xlabel("DBS rank")
ax.set_ylabel("Total UMI count")
ax.grid('on', which='major', axis='y', alpha=0.5 )
ax.grid('on', which='major', axis='x',alpha=0.5)
ax.grid('on', which='minor', axis='y', alpha=0.3)
ax.grid('on', which='minor', axis='x', alpha=0.3)
_ = plt.legend(bbox_to_anchor=(1.02, 1), title="Sample", loc='upper left')

In [ ]:
fig, ax = plt.subplots(figsize=(14,6))
ax.set_title("Read count per UMI")
for label in labels:
    temp = data_raw[data_raw["Sample"] == label].groupby("UMI", as_index=False)["ReadCount"].sum().sort_values(by="UMI",ascending=False).reset_index(drop=True)
    temp["GC"] = temp["UMI"].apply(lambda x: sum([c in {"G","C"} for c in x])/len(x)) 
    temp["ReadCount"] = temp["ReadCount"]/temp["ReadCount"].sum()
    try:
        temp.plot(ax=ax, y="ReadCount", logx=False, logy=False, label=label, alpha=0.5)
    except TypeError:
        pass
ax.set_xlabel("UMI (alphabeticaly ranked)")
ax.set_ylabel("% of total reads")
ax.grid('on', which='major', axis='y', alpha=0.5 )
ax.grid('on', which='major', axis='x',alpha=0.5)
ax.grid('on', which='minor', axis='y', alpha=0.3)
ax.grid('on', which='minor', axis='x', alpha=0.3)
_ = plt.legend(bbox_to_anchor=(1.02, 1), title="Samples", loc='upper left')

In [ ]:
temp = data_raw.groupby(["Sample", "UMI"], as_index=False, observed=True)["ReadCount"].sum().sort_values(by="UMI",ascending=False).reset_index(drop=True).copy()
temp["% GC"] = temp["UMI"].apply(lambda x: int(100*sum([c in {"G","C"} for c in x])/len(x)))
g = sns.catplot(data=temp, x="% GC", y="ReadCount", col="Sample", col_wrap=nr_cols, height=2, aspect=2, 
                 kind="point", sharey=False, capsize=0.1, estimator=np.median)

g.fig.subplots_adjust(top=0.8)
_ = g.fig.suptitle('GC bias in UMIs')

In [ ]:
temp = data_raw.groupby(["Sample", "Barcode"], as_index=False)["ReadCount"].sum().sort_values(by="Barcode",ascending=False).reset_index(drop=True).copy()
temp["% GC"] = temp["Barcode"].apply(lambda x: int(100*sum([c in {"G","C"} for c in x])/len(x))) 
g = sns.catplot(data=temp, x="% GC", y="ReadCount", col="Sample", height=2, aspect=2, col_wrap=nr_cols, 
                kind="point", alpha=0.7, sharey=False, capsize=0.1, estimator=np.median)

g.fig.subplots_adjust(top=0.8)
_ = g.fig.suptitle('GC bias in Barcodes')